# Query the data

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])

if(WEAVIATE_URL == "UPDATE_ME_WEAVIATE_URL"):
    raise Exception("Please update .env and Restart the notebook (see Restart button, next to Run All)")

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

### Helper function

In [ ]:
import json
def print_properties(item):
    print(
        json.dumps(
            item.properties,
            indent=2, sort_keys=True, default=str
        )
    )

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [ ]:
# TODO: get the Wiki collection
# wiki = 

# TODO: run a near text query, search for musical instruments, with limit 5
# response = wiki.query.

for item in response.objects:
    print_properties(item)

In [ ]:
from weaviate.classes.query import MetadataQuery

wiki = client.collections.use("Wiki")

response = wiki.query.near_text(
    query="musical instruments",
    limit=5,
    # TODO: add MetadataQuery - request distance
    # return_metadata=
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

### Autocut

Return groups of results based on the quality/distance jumps

In [ ]:
from weaviate.classes.query import MetadataQuery

wiki = client.collections.use("Wiki")

response = wiki.query.near_text(
    query="musical instruments",
    # TODO: use auto_limit instead if limit, set it to 1

    return_metadata=MetadataQuery(distance=True)
)

print(f"Returned object count: {len(response.objects)}")

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

## Filters

### Fetch with filters

In [ ]:
from weaviate.classes.query import Filter

wiki = client.collections.use("Wiki")

response = wiki.query.fetch_objects(
    limit=5,
    # TODO: filter by property title, search for something like music
    # filters=
)

for item in response.objects:
    print_properties(item)

In [ ]:
from weaviate.classes.query import Filter

response = wiki.query.fetch_objects(
    limit=5,
    # NOTE: you can use & as AND operator and | as OR operator
    filters=Filter.by_property("title").like("m*ic") & Filter.by_property("title").not_equal("music")
)

for item in response.objects:
    print_properties(item)

## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [ ]:
from weaviate.classes.query import Filter

response = wiki.query.near_text(
    query="musical instruments",
    limit=5,
    filters=Filter.by_property("title").not_equal("music")
)

for item in response.objects:
    print_properties(item)

## Keyword Search

[Docs - keyword/bm25](https://weaviate.io/developers/weaviate/search/bm25)

In [ ]:
# TODO: use bm25 query, search for musical instruments, set limit to 5

# response = wiki.

for item in response.objects:
    print_properties(item)

In [ ]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    # TODO: add query properties for "text" and "title"
    # query_properties=[],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

In [ ]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    query_properties=["text", "title^3"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [ ]:
# TODO: use hybrid query, search for musical instruments, set alpha to 0.7, and limit to 5
# response = wiki.

for item in response.objects:
    print_properties(item)

### Hybrid - select properties

In [ ]:
response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    # TODO: add query properties for "title"
)

for item in response.objects:
    print_properties(item)

### Hybrid - Explain score

In [ ]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    query_properties=["title"],
    return_metadata=MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)
    print(item.metadata.explain_score)

## Close the client

In [ ]:
client.close()